In [1]:
# python 3.7.12
# %pip install tensorflow==2.6.4
# %pip install keras==2.6.0
# %pip install matplotlib==3.5.2
# %pip install numpy==1.21.6
# %pip install pillow==9.1.1
# %pip install scikit-learn==1.0.2
# %pip install opencv-python==4.5.4.60
# %pip install pandas==1.3.5



import six
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from PIL import Image
import numpy as np
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #In case there are broken png, ignore them
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

     -------------------------------------- 429.2/429.2 MB 3.9 MB/s eta 0:00:00
     --------------------------------------- 13.2/13.2 MB 31.2 MB/s eta 0:00:00
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.7/2.7 MB 18.9 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached clang-5.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
     ---------------------------------------- 4.3/4.3 MB 67.9 MB/s eta 0:00:00
     ---------------------------------------- 430.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.


     ---------------------------------------- 3.3/3.3 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 7.1/7.1 MB 41.5 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     --------------------------------------- 34.1/34.1 MB 34.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 35.1/35.1 MB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     --------------------------------------- 10.0/10.0 MB 30.4 MB/s eta 0:00:00
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
Note: you may need to restart the kernel to use updated packages.


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
validation_split=0.2
image_data_generator = ImageDataGenerator(
                              rescale=1./255,
                              validation_split=validation_split)

# Images for training
train_generator = image_data_generator.flow_from_directory(
                        "./input/varroa-and-bees/bees_pictures/train",
                        batch_size= 128,
                        class_mode= 'categorical',
                        target_size=(160, 280), 
                        color_mode= 'rgb',
                        subset= 'training')

# Images for validation
validation_generator = image_data_generator.flow_from_directory(
                             "./input/varroa-and-bees/bees_pictures/train",
                             batch_size= 128,
                             class_mode= 'categorical',
                             target_size=(160, 280), 
                             color_mode= 'rgb',
                             subset= 'validation')
#target_size= (160, 280), 

In [ ]:
image_data_generator = ImageDataGenerator(rescale=1./255)
evaluation_generator = image_data_generator.flow_from_directory(
                                        "../input/varroa-and-bees/bees_pictures/test", 
                                        batch_size=128,
                                        class_mode='categorical',
                                        target_size=(160, 280), 
                                        color_mode='rgb')

In [ ]:
tf.config.threading.set_inter_op_parallelism_threads(1)
# learning_rate_decay = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)
image_width = 280
image_height = 160
batch_size = 128

validation_split = 0.1
initial_learning_rate = 0.045
evaluation_steps = 10

In [ ]:
img_input = tf.keras.Input(shape=(image_height, image_width, 3))

# Add convolutional layers
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

# Add dense layers
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

# Create final model
model = tf.keras.models.Model(inputs=img_input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, clipvalue=2.0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
def step_decay(epoch, lr):
    drop = 0.04
    epochs_drop = 2.0
    lrate = lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


all_files = glob('../input/varroa-and-bees/bees_pictures/train/*/*.png')

num_validation = len(all_files) * validation_split
num_train = len(all_files) - num_validation

validation_steps = int(num_validation / batch_size)
steps_per_epoch = int(num_train / batch_size)
num_test = len(all_files)

history = model.fit(train_generator,
          validation_data=validation_generator,
          epochs=1,
          steps_per_epoch=30,
          validation_steps=validation_steps)#,
#           callbacks=[early_stopping, learning_rate_decay])
model.save("./models/model.h5")

In [ ]:
models = [load_model("./models/model.h5"), load_model("./models/model (1).h5"), load_model("./models/model (2).h5")]

In [ ]:
numb = 0
predictions = []
for model in models:
    numb+=1
    print("/n/nMODEL "+ str(numb) +" :")
    predictions.append(model.predict(evaluation_generator))
    print(predictions)
    _, test_accuracy = model.evaluate(evaluation_generator, steps=evaluation_steps)
    print('Test accuracy: ' + str(round(test_accuracy * 100., 1)) + ' %')

In [ ]:
numb = 0
labels = ["Healthy", "Infected", "None"]
xyz = os.listdir('/kaggle/input/varroa-and-bees/bees_pictures/test/healthy')
print("there is " + str(len(xyz)) + " healthy bees")
xyz = os.listdir('/kaggle/input/varroa-and-bees/bees_pictures/test/infected')
print("there is " + str(len(xyz)) + " infected bees")
for prediction in predictions:
    numblab = [0,0,0,0,0]
    numb+=1
    print("/n/nMODEL "+ str(numb) +" :")
    for p in prediction:
        highest_probability_index = np.argmax(p)
        truncated_probablity = np.float64(np.round(p,8))
        if np.max(p) < .6:
            highest_probability_index = len(labels)-1
        numblab[highest_probability_index] += 1
    numb2 = 0
    for label in labels:
        print(label + " has " + str(numblab[numb2]) + " predictions")
        numb2 += 1

In [ ]:
Image.open('./input/bees_pictures/train/infected/2017-08-28_09-30-00-1_500_dirty_glass.mp4-bee_id_6698-165-1.png')

In [ ]:
model.predict(np.array(Image.open('./input/bees_pictures/train/infected/2017-08-28_09-30-00-1_500_dirty_glass.mp4-bee_id_6698-165-1.png')))